In [ ]:
import csv
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from StringIO import StringIO

sc = pyspark.SparkContext('local[*]')
sqlContext = SQLContext(sc)

In [ ]:
# CSV reader from Databricks simplifies loading CSV but is not available
#df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('cars.csv')

In [ ]:
textRDD = sc.textFile("data/sf-crime/train.csv.bz2")

textRDD.count()

In [ ]:
# remove header
header = textRDD.first()
textRDD = textRDD.filter(lambda line: not line == header)

In [ ]:
# The schema is encoded in a string.
fieldNames = header.split(',')

fields = [StructField(field_name, StringType(), True) for field_name in fieldNames]
fields[0].dataType = TimestampType()
fields[7].dataType = FloatType()
fields[8].dataType = FloatType()

schema = StructType(fields)

In [ ]:
from datetime import *
from dateutil.parser import parse

# parse each csv line (fields may contain ',') and split into tuples
tupleRDD = textRDD \
    .map(lambda line: next(csv.reader(StringIO(line)))) \
    .map(lambda x: (parse(x[0]), x[1], x[2], x[3], x[4], x[5], x[6], float(x[7]), float(x[8])))
         
crime_df = sqlContext.createDataFrame(tupleRDD, schema)
crime_df.groupBy('PdDistrict').count().show()